In [1]:
import matplotlib
from matplotlib import pyplot as plt
import numpy as np

# plt.plot(np.arange(10))

In [2]:
from glob import glob
import os
import sys
# sys.path.insert(0, "/home/delia/adversarial")
import re
# from adversarial.core.AdversarialModel import load_model
# from adversarial.core.argo.core.ArgoLauncher import ArgoLauncher
# from adversarial.core.transform.transform import load_network_with_dummy_x
from core.AdversarialModel import load_model
from core.argo.core.ArgoLauncher import ArgoLauncher
from core.transform.transform import load_network_with_dummy_x
from core.argo.core.TFDeepLearningModel import load_network
from core.argo.core.utils.argo_utils import load_class

# from adversarial.datasets.Dataset import Dataset
from datasets.Dataset import Dataset
import tensorflow as tf
import numpy as np
import pandas as pd
from functools import partial
import collections

pd_csv_kwargs = {
    'sep' : '\t'
}

def all_same(items):
    return all(x == items[0] for x in items)

def check_adv_matches(matches, base_dir):
    trimmed_matches = [match.split(base_dir)[1].strip('/') for match in matches]
    ds_strings, net_strings, transf_strings, attack_strings = zip(*[m.split("/")[:4] for m in trimmed_matches])
    if not all_same(ds_strings):
        raise Exception("ds_strings are not all the same, check your regular expression, found: {:}".format(set(ds_strings)))
    if not all_same(attack_strings):
        raise Exception("attack_string are not all the same, check your regular expression, found: {:}".format(set(attack_strings)))
    
    net_transf_strings_grouped = collections.defaultdict(list)

    for net_string, transf_string in zip(net_strings, transf_strings):
        net_transf_strings_grouped[net_string].append(transf_string)
    
    return  list(set(ds_strings))[0], net_transf_strings_grouped, list(set(attack_strings))[0]


# ds_dir = "FashionMNIST-st0" 
ds_dir = "BTSC"

# base_dir = "/data/thor_hd1/small_adv_new_experiments"
base_dir = "/data1/small_contractive-J2_experiments/"
net_dir = "FF-cCE-st0-stp*-r0"
# transf_dir = "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0"
# transf_dir = "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*_GDd128rSCR-n*-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0"
transf_dir = "recVAE-*"
# transf_dir = "recVAE-cELBO_b1.0_wuW5-s10-d0-st1-stp0.1-bs128-trA_lr0.0001_bo0.9_bt0.999-cGN100.0-ne_CNo32,32k3,3s2,2_GDd128rSCR_n2_sm0.01_sc0.0-nd_D1568_BR_CNTo32,1k3,3s2,2_GDdmc0.01-cpS-aR-wix-bic0.1-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0"

attack_dir = "advCW_n100*"

dataset_str="test"
# eps=0.1
eps = 0.01
# param_specs=[('d',), ('stp',)] #, ('ne', 'sm')]
param_specs=[('d',), ('stp',) , ('ne', 'sm')]

adv_folder="adversarial"
# outdirname="/data2/collected_adv_data"
outdirname="/data1/delia_experiments/collected_adv_data"


log_name = "test-eps{:}".format(eps)
log_file = log_name+'.npy'
dir_list = [base_dir, ds_dir, net_dir, transf_dir, attack_dir, adv_folder, log_file]
matches = glob(os.path.join(*dir_list))
if len(matches)==0:
    raise ValueError("No file found matching the provided regexpr: `{:}`".format(os.path.join(*dir_list)))

ds_string, net_transf_strings, attack_string = check_adv_matches(matches, base_dir)



W1220 12:39:29.297142 140575383492416 deprecation_wrapper.py:119] From /data1/env/tf1.14.0/lib/python3.6/site-packages/sonnet/python/custom_getters/restore_initializer.py:27: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W1220 12:39:30.091002 140575383492416 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W1220 12:39:31.061160 140575383492416 deprecation_wrapper.py:119] From /home/deliaed/adversarial/core/argo/core/hooks/ArgoHook.py:25: The name tf.train.SessionRunHook is deprecated. Please use tf.estimator.SessionRunHook instead.

W1220 12:39:31.213542 140575383492416 deprecation_wrapper.py:119] From /home/deliaed/advers

In [3]:
net_transf_strings

defaultdict(list,
            {'FF-cCE-st0-stp0.0-bs32-trA_lr0.0001_bo0.9_bt0.999-cGN100.0-nC32o3,3_BN_P2s2_DOr0.3_C32o3,3_BN_P2s2_DOr0.3_C32o3,3_BN_P2s2_DOr0.3_C32o3,3_BN_P2s2_DOr0.3_D500_D62-cpS-aR-wix-bic0.1-wrLtwo0.001-brLtwo0.001-r0': ['recVAE-cELBO_b1.0_wuW5-s10-d0-st1-stp0.1-bs128-trA_lr0.0001_bo0.9_bt0.999-cGN100.0-ne_CNo32,32k3,3s2,2_GDd128rSCR_n2_sm0.01_sc0.0-nd_D1568_BR_CNTo32,1k3,3s2,2_GDdmc0.01-cpS-aR-wix-bic0.1-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0',
              'recVAE-cELBO_b1.0_wuW5-s10-d0-st1-stp0.1-bs128-trA_lr0.0001_bo0.9_bt0.999-cGN100.0-ne_CNo32,32k3,3s2,2_GDd128rSCR_n2_sm0.1_sc0.0-nd_D1568_BR_CNTo32,1k3,3s2,2_GDdmc0.01-cpS-aR-wix-bic0.1-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0',
              'recVAE-cELBO_b1.0_wuW5-s10-d0-st1-stp0.1-bs128-trA_lr0.0001_bo0.9_bt0.999-cGN100.0-ne_CNo32,32k3,3s2,2_GDd128rSCR_n2_sm0.001_sc0.0-nd_D1568_BR_CNTo32,1k3,3s2,2_GDdmc0.01-cpS-aR-wix-bic0.1-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0',
              'recVAE-cELBO_b1.0_wuW5-s10

In [4]:

# for transf_string in transf_strings:
# # plot with fixed transf


In [5]:
net_string = list(net_transf_strings.keys())[0]
transf_strings = net_transf_strings[net_string]
transf_string = transf_strings[0]
# import pdb; pdb.set_trace()

# loading the Adversarial Model conf
conf_file = os.path.join(*[base_dir, ds_string, net_string, transf_string, attack_string, "experiment.conf"])
# model, dataset = load_model(conf_file, model_class_base_path="core")
dataset_conf, model_parameters, config = ArgoLauncher.process_conf_file(conf_file)
transform_tuple = model_parameters["transform"]
# loading the Adversarial Model conf
dataset = Dataset.load_dataset(dataset_conf)


In [6]:
labels = dataset.get_raw_labels("test")[:1000]
n, bins, patches = plt.hist(x=labels, bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)

In [7]:
# CREATE SESSION
tf.set_random_seed(1000)


import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]='1'
sess_config = tf.ConfigProto()
sess_config.gpu_options.allow_growth=True



In [8]:
# from word_embedding.test.core.clustering.utils import riemannian_dist
def rough_sqdist_numerics_diagzero(sqdistances):

    np.fill_diagonal(sqdistances, 0.)
    min_dist = np.min(sqdistances)
    max_dist = np.max(sqdistances)

    if min_dist < 0.:
        sqdistances += -np.min(sqdistances)
        np.fill_diagonal(sqdistances, 0.)

    return sqdistances

def check_sqdist_numerics(sqdistances):
    min_dist = np.min(sqdistances)
    max_dist = np.max(sqdistances)

    all_ok = min_dist >= 0 or -min_dist < DISTABSTOL  # ((max_dist-min_dist)/max_dist - 1 < DISTRELTOL and -min_dist < DISTABSTOL)
    assert all_ok, "sqdistances are wrong, found: min {:} ,  max {:}".format(min_dist, max_dist)

    if min_dist < 0.:
        sqdistances += -np.min(sqdistances)

    return sqdistances


def riemannian_dist(xp, xq, g_matrix):

    Ixp = np.matmul(g_matrix, xp.T)
    sqnormp = np.sum(xp * Ixp.T, axis=1)

    Ixq = np.matmul(g_matrix, xq.T)
    sqnormq = np.sum(xq * Ixq.T, axis=1)

    sqdistances = sqnormp.reshape(-1, 1) + sqnormq.reshape(1, -1) - 2 * np.matmul(xp, Ixq)

    # # temporarily remove this check put a rough compliance function
    # sqdistances = check_sqdist_numerics_diagzero(sqdistances)
    sqdistances = rough_sqdist_numerics_diagzero(sqdistances)

    
    distances = np.sqrt(sqdistances)

    return distances


def nwwild(string, field_spec):
    l = len(field_spec)
    
    if l==0 or l>2:
        raise ValueError("Not implemented tuple length `{:}`, found field spec `{:}`".format(l, field_spec))
    
    val = get_field(string, field_spec)
    
    if l==1:
        fsplit = '-'+field_spec[0]
    elif l==2:
        fsplit ='_'+field_spec[1]
    
    ssplit = fsplit+val
    
    
    if ssplit in string:
        pre_stp, post_stp = string.split(ssplit)
        return pre_stp+fsplit+'[W]'+post_stp
    else:
        return None

    
def name_with_wildcards(strings, param_specs):
    for ps in param_specs:
        strings = list(map(partial(nwwild, field_spec=ps), strings))
        strings = [s for s in strings if s is not None]
#         import pdb; pdb.set_trace()
    if not all_same(strings):
        raise Exception("strings not all the same, found: ```{:}```".format(set(strings)))
    
    return list(set(strings))[0]

def get_param_vals(string, param_specs):
    param_vals = []
    for ps in param_specs:
        pv = get_field(string, ps)
        param_vals.append(pv)
    return param_vals

def get_field(string, field_spec):
    l = len(field_spec)
    
    if l==0 or l>2:
        raise ValueError("Not implemented tuple length `{:}`, found field spec `{:}`".format(l, field_spec))
    
    m = re.search('(-|^)' + field_spec[0] + '([\._A-Za-z0-9\,]+)' + '(-|$)', string)
    if m is None:
        ss1 = '0'
    else:
        ss1 = m.group(2)
    
    if l==1:
        return ss1
    
    m = re.search('(_|^)' + field_spec[1] + '([\.A-Za-z0-9\,]+)' + '(_|$)', ss1)

    if m is None:
        ss2 = '0'
    else:
        ss2 = m.group(2)
    
    return ss2

def spec_name(param_spec):
    return "_".join(param_spec)

def extract_one_point_info(base_dir, ds_string, net_string, transf_string, attack_string, sess_config, dataset_str, eps):
    tf.reset_default_graph()
    sess = tf.Session(config = sess_config)
    
    # loading the Adversarial Model conf
    conf_file = os.path.join(*[base_dir, ds_string, net_string, transf_string, attack_string, "experiment.conf"])
    # model, dataset = load_model(conf_file, model_class_base_path="core")
    dataset_conf, model_parameters, config = ArgoLauncher.process_conf_file(conf_file)
    transform_tuple = model_parameters["transform"]
    # loading the Adversarial Model conf

    dataset = Dataset.load_dataset(dataset_conf)
    x_adv = np.load(os.path.join(*[base_dir, ds_string, net_string, transf_string, attack_string, "adversarial", dataset_str+"-eps{:}.npy".format(eps)]))
    x_clean = dataset.get_raw_elements(dataset_str)[:1000]
    # # this checks the norms are what they should be
    # # np.linalg.norm(x_clean.reshape(-1,784)-x_adv.reshape(-1,784), ord=np.inf, axis=1)
    # loading the transformation
    transform_name, transform_kwargs = transform_tuple
    is_training=False
    transf_conf_file = transform_kwargs['conf_file']
    global_step = transform_kwargs['global_step']
    x_shape = (None,) + dataset.x_shape

    x_ph = tf.placeholder(tf.float32, shape=x_shape, name='input')
    
    t_network = load_network_with_dummy_x(x_ph,
                                        sess,
                                        is_training=is_training,
                                        conf_file=transf_conf_file,
                                        global_step=global_step,
                                        base_path="vae.core")
    # loading the transformation

    encoder_module = t_network.encoder_module
    decoder_module = t_network.decoder_module
    
    enc = encoder_module(x_ph)
    
    if transform_name=='vae':
        mu = enc.mean()
    elif transform_name=='ae':
        mu = enc
    else:
        raise Exception("not supported")
    
    mu_np = sess.run(mu, feed_dict={x_ph:x_clean})
    mu_star_np = sess.run(mu, feed_dict={x_ph:x_adv})
    norm_np = np.linalg.norm(mu_np, axis=1)
    norm_star_np = np.linalg.norm(mu_star_np, axis=1)
    adiff = norm_star_np - norm_np
    costh = np.sum(mu_np * mu_star_np, axis=1)/(norm_np*norm_star_np)
    
    onevec = np.ones((1, mu_np.shape[1]))
    norm_onevec = np.linalg.norm(onevec)
    costh_mu = np.sum(mu_np * onevec, axis=1)/(norm_np*norm_onevec)
    costh_mu_adv = np.sum(mu_star_np * onevec, axis=1)/(norm_star_np*norm_onevec)
    nratio = norm_star_np/norm_np
    
    ldim = mu_np.shape[1]
    euclidean_dist = partial(riemannian_dist, g_matrix=np.eye(ldim))
    
    distances_mu_np = euclidean_dist(mu_np, mu_np)
    distances_mu_star_np = euclidean_dist(mu_star_np, mu_star_np)
#     import pdb; pdb.set_trace()
    diameter_mu_set = np.max(distances_mu_np)
    diameter_mu_star_set = np.max(distances_mu_star_np)

# exclude the diagonal in case you want to do average of distances
#     mask_out_of_diag = np.ones(distances_mu_np.shape, dtype=bool)
#     np.fill_diagonal(mask_out_of_diag, 0)
#     avgdist_mu_set = np.mean(distances_mu_np[mask_out_of_diag])
#     avgdist_mu_star_set = np.mean(distances_mu_star_np[mask_out_of_diag])
    
    centroid_mu_set = np.mean(mu_np, axis=0)
    centroid_mu_star_set = np.mean(mu_star_np, axis=0)
    
    cdists_set = euclidean_dist(mu_np, centroid_mu_set.reshape(1, -1))
    cdists_adv_set = euclidean_dist(mu_star_np, centroid_mu_star_set.reshape(1, -1))
    
    ndiffs_np = np.linalg.norm(mu_star_np-mu_np, axis=1)
    ndiff_avg = np.mean(ndiffs_np)
    ndiff_std = np.std(ndiffs_np)
    
    adiff_avg = np.mean(adiff)
    adiff_std = np.std(adiff)
    nratio_avg = np.mean(nratio)
    nratio_std = np.std(nratio)
    costh_avg = np.mean(costh)
    costh_std = np.std(costh)
    costh_mu_avg = np.mean(costh_mu)
    costh_mu_std = np.std(costh_mu)
    costh_mu_adv_avg = np.mean(costh_mu_adv)
    costh_mu_adv_std = np.std(costh_mu_adv)

    norms_avg = np.mean(norm_np)
    norms_std = np.std(norm_np)
    
    norms_adv_avg = np.mean(norm_star_np)
    norms_adv_std = np.std(norm_star_np)

    cdists_set_mean = np.mean(cdists_set)
    cdists_set_std = np.std(cdists_set)
    cdists_adv_set_mean = np.mean(cdists_adv_set)
    cdists_adv_set_std = np.std(cdists_adv_set)
    
    sess.close()
#     tf.reset_default_graph()
    return [adiff_avg, adiff_std,
            norms_avg, norms_std,
            norms_adv_avg, norms_adv_std,
            ndiff_avg, ndiff_std,
            nratio_avg, nratio_std,
            costh_avg, costh_std,
            costh_mu_avg, costh_mu_std,
            costh_mu_adv_avg, costh_mu_adv_std,
            cdists_set_mean, cdists_set_std,
            cdists_adv_set_mean, cdists_adv_set_std,
            diameter_mu_set,
            diameter_mu_star_set,
           ]


    

In [9]:
param_names = list(map(spec_name, param_specs))
param_names

['d', 'stp', 'ne_sm']

In [10]:
string = transf_string

In [11]:
get_field(string, ('ne', 'sm'))

'0.01'

In [12]:
get_param_vals(string, param_specs)

['0', '0.1', '0.01']

In [13]:
'_sm' in string

True

In [14]:
m = re.search('(-|^)' + 'ne' + '[\._A-Za-z0-9\,]+'+ '([\_])'+ 'sm' + '(-|$)', string)

In [15]:
print(string)
print(nwwild(string, param_specs[0]))
print(nwwild(string, param_specs[1]))


recVAE-cELBO_b1.0_wuW5-s10-d0-st1-stp0.1-bs128-trA_lr0.0001_bo0.9_bt0.999-cGN100.0-ne_CNo32,32k3,3s2,2_GDd128rSCR_n2_sm0.01_sc0.0-nd_D1568_BR_CNTo32,1k3,3s2,2_GDdmc0.01-cpS-aR-wix-bic0.1-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0
recVAE-cELBO_b1.0_wuW5-s10-d[W]-st1-stp0.1-bs128-trA_lr0.0001_bo0.9_bt0.999-cGN100.0-ne_CNo32,32k3,3s2,2_GDd128rSCR_n2_sm0.01_sc0.0-nd_D1568_BR_CNTo32,1k3,3s2,2_GDdmc0.01-cpS-aR-wix-bic0.1-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0
recVAE-cELBO_b1.0_wuW5-s10-d0-st1-stp[W]-bs128-trA_lr0.0001_bo0.9_bt0.999-cGN100.0-ne_CNo32,32k3,3s2,2_GDd128rSCR_n2_sm0.01_sc0.0-nd_D1568_BR_CNTo32,1k3,3s2,2_GDdmc0.01-cpS-aR-wix-bic0.1-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0


In [16]:
# import pdb; pdb.set_trace()
outputname = name_with_wildcards(transf_strings, param_specs)


In [17]:
outputname

'recVAE-cELBO_b1.0_wuW5-s10-d[W]-st1-stp[W]-bs128-trA_lr0.0001_bo0.9_bt0.999-cGN100.0-ne_CNo32,32k3,3s2,2_GDd128rSCR_n2_sm[W]_sc0.0-nd_D1568_BR_CNTo32,1k3,3s2,2_GDdmc0.01-cpS-aR-wix-bic0.1-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0'

In [19]:

def collect_stats_over_transf(ds_string, net_string, transf_strings, attack_string, param_specs, dataset_str, eps, outdirname):

    param_names = list(map(spec_name, param_specs))
    all_fields = param_names + ["adiff_avg", "adiff_std",
                                "norm_avg", "norm_std",
                                "norm_adv_avg", "norm_adv_std",
                                "ndiff_avg", "ndiff_std",
                                "nratio_avg", "nratio_std",
                                "costh_avg", "costh_std",
                                "costh_mu_avg", "costh_mu_std",
                                "costh_mu_adv_avg", "costh_mu_adv_std",
                                "cdist_avg", "cdist_std",
                                "cdist_adv_avg", "cdist_adv_std",
                                "diameter_mu",
                                "diameter_mu_adv",]
    data = []
    for transf_string in transf_strings:
        param_vals = get_param_vals(transf_string, param_specs)

        info_values_list = extract_one_point_info(base_dir, ds_string, net_string, transf_string, attack_string, sess_config, dataset_str, eps)
        
        data.append(param_vals + info_values_list)

    sorted_data = sorted(data)

    df = pd.DataFrame(sorted_data, columns=all_fields)
#     import pdb; pdb.set_trace()
    outputname = name_with_wildcards(transf_strings, param_specs)

    out_dir_list = [outdirname, ds_string, attack_string, net_string]
    outpath = os.path.join(*out_dir_list)
    os.makedirs(outpath, exist_ok=True)
    outpath = os.path.join(outpath,  outputname+'-ds{:}-eps{:}.csv'.format(dataset_str, eps))
    df.to_csv(outpath, index=False, float_format='%.6g', **pd_csv_kwargs)



In [20]:
def collect_stats_per_dataset(ds_dir, net_dir, transf_dir, attack_dir, eps, base_dir, outdirname):
    
    dataset_str="test"
#     param_specs=[('d',), ('stp',)] #, ('ne', 'sm')]
    param_specs=[('d',), ('stp',), ('ne', 'sm')]

    adv_folder="adversarial"

    log_name = "test-eps{:}".format(eps)
    log_file = log_name+'.npy'
    dir_list = [base_dir, ds_dir, net_dir, transf_dir, attack_dir, adv_folder, log_file]
    matches = glob(os.path.join(*dir_list))
    if len(matches)==0:
        raise ValueError("No file found matching the provided regexpr: `{:}`".format(os.path.join(*dir_list)))

    ds_string, net_transf_strings_grouped, attack_string = check_adv_matches(matches, base_dir)

    # for net_string in net_strings:
    # # plot with fixed net
    for net_string in net_transf_strings_grouped:
        transf_strings = net_transf_strings_grouped[net_string]
        collect_stats_over_transf(ds_string, net_string, transf_strings, attack_string, param_specs, dataset_str, eps, outdirname)


In [32]:
# base_dir = "/data1/small_adv_new_experiments"
# base_dir = "/data/wonderwoman_hd1/small_adv_new_experiments"
base_dir = "/data1/small_contractive-J2_experiments/"
# ds_dirs = ["MNIST-c-st0", "FashionMNIST-st0"] #"SVHN-st0"]
# ds_dirs = ["MNIST-c-st0"]
ds_dirs = ['BTSC']
#"BTSC", 

# outdirname="/data2/collected_adv_data"
outdirname="/data1/delia_experiments/collected_adv_data"


# net_dirs_dict = {
    
#     "FashionMNIST-st0" : [
#         ("FF-cCE-st0-stp*-r0", "recAE-cL2-d*-st1-stp*-trA_lr0.0001_*_D128-nd*-wrLtwo0.01-brLtwo0.01-*", [0.1, 0.2, 0.3]),
#         ("FF-cCE-st0-stp*-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0", [0.1, 0.2, 0.3]),
#     ],
    
#     "MNIST-c-st0" : [
#         ("FF-cCE-st0-stp*-r0", "recAE-cL2-d*-st1-stp*-trA_lr0.0001_*_D128-nd*-wrLtwo0.01-brLtwo0.01-*", [0.1, 0.2, 0.3]),
#         ("FF-cCE-st0-stp*-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*-cV10*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0", [0.1, 0.2, 0.3]),
#         ("FF-cCE-st0-stp*-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*-cGN100.0*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0", [0.1, 0.2, 0.3]),
#     ],
    
#     "SVHN-st0" : [
#         ("FF-cCE-st0-stp*-r0", "recAE-cL2-d*-st1-stp*-trA_lr0.0001_*_D128-nd*-wrLtwo0.01-brLtwo0.01-*", 0.03),
#         ("FF-cCE-st0-*-wrLtwo0.001-brLtwo0.001-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0", 0.03)
#     ],
    
#     "BTSC" : [
#         ("FF-cCE-st0-stp*-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0", 0.02)
#     ],
# }

# net_dirs_dict = {
    
#     "MNIST-c-st0" : [
#         ("FF-cCE-st0-stp*-r0", "recAE-cL2-d*-st1-stp*-trA_lr0.0001_*_D128-nd*-wrLtwo0.01-brLtwo0.01-*", [0.1, 0.2, 0.3]),
#         ("FF-cCE-st0-stp*-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*-cV10*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0", [0.1, 0.2, 0.3]),
#         ("FF-cCE-st0-stp*-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*-cGN100.0*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0", [0.1, 0.2, 0.3]),
#     ],
# }

net_dirs_dict = {
    
    "BTSC" : [
        ("FF-cCE-st0-stp*-r0", "recVAE-*", 0.01)
    ],
}

attack_dir = "advCW_n100*"

# for ds_dir in ds_dirs:
#     for net_dir, transf_dir, epsilons in net_dirs_dict[ds_dir]:
#         for eps in epsilons:
#             collect_stats_per_dataset(ds_dir, net_dir, transf_dir, attack_dir, eps, base_dir, outdirname)

ds_dir = ds_dirs[0]
net_dir, transf_dir, epsilons = net_dirs_dict[ds_dir][0]
collect_stats_per_dataset(ds_dir, net_dir, transf_dir, attack_dir, epsilons, base_dir, outdirname)


ValueError: No file found matching the provided regexpr: `/data1/small_contractive-J2_experiments/BTSC/FF-cCE-st0-stp*-r0/recVAE-*/advCW_n100*/adversarial/test-eps0.1.npy`

In [28]:
def plot_grouped_curves(df, x, y, groupfield, pre_label):
    garr = [(label, df) for label, df in df.groupby(groupfield)]
    for groupval, df in garr:
        plabel = pre_label+'-'+groupfield+"{:}".format(groupval)
        xdata = df[x]
        
        pkwargs = {
            'alpha' : ALPHA,
            'label' : plabel
        }
        
        if y+'_avg' in df.columns and y+'_std' in df.columns:  
            ydata = df[y+'_avg']
            yerr = df[y+'_std']
            plt.errorbar(xdata, ydata, yerr=yerr, **pkwargs)
        else:
            ydata = df[y]
            plt.plot(xdata, ydata, **pkwargs)


nice_yaxis_names = {
    'nratio' : '$|\mu^*|/|\mu|$',
    'costh' : r'$\cos \, \theta$',
    'costh_mu' : r'$\cos \, \theta$',
    'costh_mu_adv' : r'$\cos \, \theta$',
    'adiff' : '$|\mu^*| - |\mu|$',
    'ndiff' : '$|\mu^* - \mu|$',
    'norm' : '$|\mu|$',
    'norm_adv' : '$|\mu^*|$',
    'cdist' : 'centroid dist',
    'cdist_adv' : 'centroid dist',
    'diameter_mu' : 'diameter',
    'diameter_mu_adv' : 'diameter',
}

ALPHA = 0.6

def make_ds_plot(ds_dir, csv_dirs_dict, y, eps, plot_dir):
    x = 'stp'
    nice_ds_name = ds_dir.split("-")[0]
    for net_dir, transf_dir, sm in csv_dirs_dict[ds_dir]:
        transf_csv = transf_dir+'*.csv'

        dir_list = [base_csv_dir, ds_dir, attack_dir, net_dir, transf_csv]
        matches = list(set(glob(os.path.join(*dir_list))))

        # print(matches)
        groupfield = 'd'
        plt.title(nice_ds_name+' eps={:}'.format(eps))
        plt.xlabel(x)
        plt.ylabel(nice_yaxis_names[y])
        for match in sorted(matches):
            transf_string = os.path.splitext(match.split("/")[-1])[0]
            recv = get_field(transf_string, ('rec',))
            dsv = get_field(transf_string, ('ds',))
            epsv = get_field(transf_string, ('eps',))
            if not np.isclose(eps, float(epsv)):
                continue
            df = pd.read_csv(match, **pd_csv_kwargs)
#             import pdb; pdb.set_trace()
            plot_grouped_curves(df, x, y, groupfield, recv)
    
    fields = [nice_ds_name, x, y, 'eps{:.2f}'.format(eps)]
    plt.tight_layout()
    plt.legend(loc='best')
    plt.savefig(os.path.join(plot_dir, '-'.join(fields)+'.png'))
    plt.close()

In [29]:
def make_ds_plot_in_place(ds_dir, csv_dirs_dict, y, eps, plot_dir):
#     fig = plt.figure()
    x = 'stp'
    nice_ds_name = ds_dir.split("-")[0]
#     import pdb; pdb.set_trace()
    for net_dir, transf_dir, sm in csv_dirs_dict[ds_dir]:
        transf_csv = transf_dir+'*.csv'

        dir_list = [base_csv_dir, ds_dir, attack_dir, net_dir, transf_csv]
        matches = list(set(glob(os.path.join(*dir_list))))

        # print(matches)
        groupfield = 'd'
        plt.title(nice_ds_name+' eps={:}'.format(eps))
        plt.xlabel(x)
        plt.ylabel(nice_yaxis_names[y])
        for match in sorted(matches): 
#             print(match)
            transf_string = os.path.splitext(match.split("/")[-1])[0]
            recv = get_field(transf_string, ('rec',))
            dsv = get_field(transf_string, ('ds',))
            epsv = get_field(transf_string, ('eps',))
#             import pdb; pdb.set_trace()
            if not np.isclose(eps, float(epsv)):
                continue
            df = pd.read_csv(match, **pd_csv_kwargs)
#             import pdb; pdb.set_trace()
            plot_grouped_curves(df, x, y, groupfield, recv)
    
    fields = [nice_ds_name, x, y, 'eps{:.2f}'.format(eps)]
    plt.tight_layout()
    plt.legend(loc='best')
    plt.savefig(os.path.join(plot_dir, '-'.join(fields)+'.png'))
    print(os.path.join(plot_dir, '-'.join(fields)+'.png'))
    plt.show()
#     plt.clf()
#     plt.close()

In [31]:
matplotlib.rcParams.update({'savefig.dpi': '300'})

fontsize=30
fonttitlesize=34
fontaxeslabelsize=30
fontlegendsize=24

matplotlib.rcParams.update({'font.size': fontsize})
# matplotlib.rcParams.update({'font.weight': 'bold'})
# matplotlib.rcParams['text.latex.preamble'] = [r'\boldmath']

matplotlib.rcParams.update({'legend.frameon': False})
matplotlib.rcParams.update({'legend.fontsize': fontlegendsize})
matplotlib.rcParams.update({'legend.borderaxespad': 1.0})

matplotlib.rcParams.update({'lines.linewidth': 4.0})
matplotlib.rcParams.update({'lines.markersize': 9})
matplotlib.rcParams.update({'lines.marker': 'o'})

matplotlib.rcParams.update({'errorbar.capsize': 7})
# matplotlib.rcParams.update({'errorbar.capthick': 3})

matplotlib.rcParams.update({'axes.titlesize': fonttitlesize})
# matplotlib.rcParams.update({'axes.titleweight': 'bold'})
matplotlib.rcParams.update({'axes.labelsize': fontaxeslabelsize})
# matplotlib.rcParams.update({'axes.labelweight': 'bold'})

# matplotlib.rcParams.update({'axes.labelpad': 16.0})
# matplotlib.rcParams.update({'xtick.major.pad': 10.0})
# matplotlib.rcParams.update({'ytick.major.pad': 5.0})

figsize = matplotlib.rcParams['figure.figsize']
figsize[0] = 6.4 * 2
figsize[1] = 4.8 * 2


base_csv_dir = outdirname
# base_plot_dir = "/data2/collected_adv_data_plots"
base_plot_dir = "/data1/delia_experiments/collected_adv_data_plots"

# ds_dirs = ["MNIST-c-st0", "FashionMNIST-st0"]
# ds_dirs = ["MNIST-c-st0"]
ds_dirs = ['BTSC']
# csv_dirs_dict = {
    
#     "FashionMNIST-st0" : [
#         ("FF-cCE-st0-stp*-r0", "recAE-cL2-d*-st1-stp*-trA_lr0.0001_*_D128-nd*-wrLtwo0.01-brLtwo0.01-*"),
#         ("FF-cCE-st0-stp*-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-*"),
#     ],
    
#     "MNIST-c-st0" : [
#         ("FF-cCE-st0-stp*-r0", "recAE-cL2-d*-st1-stp*-trA_lr0.0001_*_D128-nd*-wrLtwo0.01-brLtwo0.01-*"),
#         ("FF-cCE-st0-stp*-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*-cV10*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-*"),
# #         ("FF-cCE-st0-stp*-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*-cGN100.0*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0", 0.1),
#     ],
    
# #     "SVHN-st0" : [
# #         ("FF-cCE-st0-*-wrLtwo0.001-brLtwo0.001-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0", 0.015)
# #     ],
    
# #     "BTSC" : [
# #         ("FF-cCE-st0-stp*-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0", 0.02)
# #     ],
# }
# csv_dirs_dict = {
    
#     "MNIST-c-st0" : [
#         ("FF-cCE-st0-stp*-r0", "recAE-cL2-d*-st1-stp*-trA_lr0.0001_*_D128-nd*-wrLtwo0.01-brLtwo0.01-*"),
#         ("FF-cCE-st0-stp*-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*-cV10*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-*"),
# #         ("FF-cCE-st0-stp*-r0", "recVAE-cELBO_b1.0_wuW5-s10-d*-st1-stp*-trA_lr0.0001_*-cGN100.0*_GDd128-nd*-wrLtwo0.01-brLtwo0.01-r0-sh0-sx0-zsc1.0", 0.1),
#     ],
    
# }

csv_dirs_dict = {
    "BTSC" : [
        ("FF-cCE-st0-stp*-r0", "recVAE-*", 0.01)
    ],
}

# os.makedirs(base_plot_dir, exist_ok=True)
# for ds_dir in ds_dirs:
#     for y in ['nratio', 'costh', 'costh_mu', 'costh_mu_adv', 'adiff', 'ndiff', 'norm', 'norm_adv', 'cdist', 'cdist_adv', 'diameter_mu', 'diameter_mu_adv']:
# #         for eps in [0.1, 0.2 ,0.3]:
#         make_ds_plot_in_place(ds_dir, csv_dirs_dict, y, 0.01, base_plot_dir)
    
os.makedirs(base_plot_dir, exist_ok=True)
for ds_dir in ds_dirs:
    for y in ['nratio']:
        make_ds_plot(ds_dir, csv_dirs_dict, y, 0.01, base_plot_dir)


In [23]:


# for label, df in df.groupby(groupfield):
    
#     plt.errorbar(xdata, ydata, yerr=yerr, label='VAE-'groupfield+"{:}".format(label))
    

# plt.tight_layout()
# plotfilename = self._plotfilename
# if suffix is not None:
#     plotfilename+=suffix
# plt.savefig(plotfilename+".png")
# plt.close()


In [24]:
garr = [(label, df) for label, df in df.groupby(groupfield)]

NameError: name 'df' is not defined

In [ ]:
garr[0][1]

In [ ]:
# y = 'adiff'
# y = 'costh'
y = 'nratio'


for groupval, df in garr:
    xdata = df['stp']
    ydata = df[y+'_mu']
    yerr = df[y+'_std']
    plt.errorbar(xdata, ydata, yerr=yerr, label='VAE-'+groupfield+"{:}".format(groupval))
plt.legend()

In [ ]:
a

In [ ]:
model._transform_feedable

In [ ]:
x_adv.shape

In [ ]:
dataset.test_set_x.shape

In [ ]:
28*28

In [ ]:
sess.run(out, feed_dict={x_placeholder:x_clean})

In [ ]:
sess.run(model._logits, feed_dict={model.x: x_clean})
sess.run(model._transform_module.enc.sample(), feed_dict={model.x: x_clean})

In [ ]:
field_lpsch_avg = ""
field_lpsch_std = ""
field_costh_avg = ""
field_costh_std = ""

all_fields = [param_name, 'epoch', field_train, field_val, field_test]

data = []
for match in matches:
    x = get_ds_field_value(match, pre_ds_dir, post_ds_dir)
    epmax, y_val = read_max_value(match, field_val, 'epoch')
    y_train = read_value(match, 'epoch', epmax, field_train)
    y_test = read_value(match, 'epoch', epmax, field_test)
    data.append((tryfloat(x), epmax, y_train, y_val, y_test))

sorted_data = sorted(data, key=key_sort)

df = pd.DataFrame(sorted_data, columns=all_fields)

outpath = os.path.join(outdirname, pre_ds_dir+'W'+post_ds_dir, net_string)
os.makedirs(outpath, exist_ok=True)
outpath = os.path.join(outpath, "collected_"+log_file)
df.to_csv(outpath, index=False, float_format='%.6g', **pd_csv_kwargs)

